In [14]:
import graphviz as gv
# Hab das mal bei mir installiert für die Darstellung
import os

import csv, re
import numpy as np
from typing import List

In [15]:
rows = []
with open("../../example-structure.csv", "r") as f:
    reader = csv.reader(f)
    rows = [row for row in reader if row != []]
print(rows)

[['Semaphore', ' 12', ' 0'], ['Semaphore', ' 24', ' 0'], ['Semaphore', ' 13', ' 0'], ['Semaphore', ' 36', ' 0'], ['Semaphore', ' 24', ' 0'], ['Semaphore', ' 46', ' 0'], ['Semaphore', ' 65', ' 0'], ['Semaphore', ' 5ab', ' 0'], ['Semaphore', ' 5ba', ' 1'], ['Semaphore', ' 51', ' 1'], ['Semaphore', ' 46', ' 0'], ['Semaphore', ' 65', ' 0'], ['Semaphore', ' 42', ' 0'], ['Mutex', ' 234'], ['Mutex', ' 345'], ['Activity', ' 1', ' 8', ' 51', ' 12;13', ' x'], ['Activity', ' TEST', ' 8', ' 51:42;13', ' 12;13', ' x'], ['Activity', ' 2', ' 8', ' 12', ' 24', ' 234;345'], ['Activity', ' 3', ' 8', ' 13', ' 36', ' 234'], ['Activity', ' 4', ' 8', ' 24', ' 46', ' 234'], ['Activity', ' 5a', ' 8', ' 65;5ba', ' 5ab', ' x'], ['Activity', ' 5b', ' 8', ' 5ab', ' 51;5ba', ' x'], ['Activity', ' 6', ' 8', ' 46;36', ' 65', ' x'], ['Task', ' 1', ' 1'], ['Task', ' 2', ' 2'], ['Task', ' 3', ' 3'], ['Task', ' 4', ' 4'], ['Task', ' 5', ' 5a;5b'], ['Task', ' 6', ' 6']]


In [21]:
class Semaphore():
    def __init__(self, name, value=0):
        self._name = name
        self._value = value
class Activity():
    def __init__(self, name, duration, incoming_semaphores, outgoing_semaphores):
        self._name = name
        self._duration = duration
        self._incoming_semaphores = incoming_semaphores
        self._outgoing_semaphores = outgoing_semaphores
class Task():
    def __init__(self, task_name, activity_list: List):
        self._name = task_name
        self._activities = activity_list
    def get_name(self) -> str:
        return self._name
    @property
    def get_activities(self) -> List:
        return self._activities

class Mutex():
    def __init__(self, activity_list: List):
        self._activity_list = activity_list
        self._reserved = False

_semaphores = []
_activities = []
_tasks = []
_mutexes = []

def find_activities( activities) -> List[Activity]:
    found_activities = []
    for activty in activities.split(";"):
        found_activities.append(find_in_array(_activities, activty))
    return found_activities
def parse(rows):
        global _semaphores, _activities, _tasks, _mutexes
        for object in rows:
            if object[0] == "Task":
                included_activities = find_activities(object[2])
                _tasks.append(Task(object[1], included_activities))
                print("Task created")
            elif object[0] == "Activity":
                incoming_semaphores = find_semaphores(object[3])
                outgoing_semaphores = find_semaphores(object[4])
                _activities.append(Activity(object[1], object[2], incoming_semaphores, outgoing_semaphores))
                print("Activity created")
            elif object[0] == "Semaphore":
                _semaphores.append(Semaphore(object[1].strip(), object[2].strip()))
                print("Semaphore created")
            #elif object[0] == "Mutex":
            #    connected_semaphores = find_semaphores(object[1])
            #    _mutexes.append(Mutex(connected_semaphores))
            #    print("Mutex created")

def find_semaphores( semaphores) -> List[Semaphore]:
    found_semaphores = []
    for semaphore in semaphores.split(";"):
        if semaphore.count(":") > 0:
            found_semaphore_relation = []
            # Do something if argument is a numpy array
            for related_semaphore in semaphore.split(":"):
                found_semaphore_relation.append(find_in_array(_semaphores, related_semaphore.strip()))
            found_semaphores.append(found_semaphore_relation)
        else:
            # Do something if argument is a string
            found_semaphores.append(find_in_array(_semaphores, semaphore))
    return found_semaphores

def find_in_array(array, name):
    return [element for element in array if element._name == name.strip()]


parse(rows)
print("Finihsed")

Semaphore created
Semaphore created
Semaphore created
Semaphore created
Semaphore created
Semaphore created
Semaphore created
Semaphore created
Semaphore created
Semaphore created
Semaphore created
Semaphore created
Semaphore created
Activity created
Activity created
Activity created
Activity created
Activity created
Activity created
Activity created
Activity created
Task created
Task created
Task created
Task created
Task created
Task created
Finihsed


In [36]:
# https://graphviz.readthedocs.io/en/stable/manual.html

# Bildname kommt hinten in die Klammer rein
# Die dot Variable immer vor so einem Block ausführen
dot = gv.Digraph(comment='TestGraph')

for task in _tasks:
    task_name = task.get_name()
    #act_list = ' '.join(task.get_activities())   
    dot.node(name=task_name,shape='record', style='filled', fillcolor='green', label=f"{task_name}|Activity")
    pass


#dot.node('A', shape='record', style='filled', fillcolor='green', label='{Up|Down}')
#dot.node('B', shape='record', style='filled', fillcolor='white', label='{Up|Down}')
task1 = _tasks[0].get_name()
task2 = _tasks[1].get_name()
dot.edge(f'{task1}', f'{task2}', label='15', arrowhead='', color='green')

dot.render('rectangle_arrow', view=True, format='svg')
os.remove("rectangle_arrow")

In [ ]:
# Erstellen der Tasks / Aktivitäten 
# Verbinden durch Semaphoren
# Erstellen der Mutexes und dazugehörigen Verbindungen
#
#